In [1]:
#this is the script which lods the decoders and model from the previous trained data
#evreything is not ellegant yet so no function that does it all

In [2]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from collections import defaultdict
import pickle
from pickle import dump
from pickle import load


%matplotlib inline

In [3]:
data = pd.read_csv('fifa21_validate.csv')

In [4]:
pd.set_option('display.max_columns', None)
data.head(5)

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,Right,7,"Feb 8, 2018","Jun 30, 2021",€975K,€5K,€0,"Jun 30, 2021 On Loan",230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34,48+2,48+2,48+2,48+0,47+0,47+0,47+0,48+0,48+2,48+2,48+2,50+2,51+2,51+2,51+2,50+2,59+2,61+2,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,Right,5,"Jan 20, 2020",NaN,€1.2M,€3K,€2.2M,2020 ~ 2020,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4 ★,3★,High,Low,1 ★,74,67,62,74,24,57,12,64+2,64+2,64+2,68+0,68+0,68+0,68+0,68+0,68+2,68+2,68+2,67+2,61+2,61+2,61+2,67+2,49+2,47+2,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,Right,17,"Nov 28, 2018",NaN,€120K,€500,€249K,2018 ~ 2020,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2 ★,1★,Medium,Medium,1 ★,56,52,53,55,26,53,3,18+2,18+2,18+2,15+0,17+0,17+0,17+0,15+0,17+2,17+2,17+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,Right,23,"Jan 10, 2020",NaN,€160K,€500,€464K,2020 ~ 2022,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3 ★,2★,Medium,Medium,1 ★,58,34,47,52,53,59,6,46+2,46+2,46+2,47+0,46+0,46+0,46+0,47+0,47+2,47+2,47+2,49+2,49+2,49+2,49+2,49+2,53+2,54+2,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,Right,5,"Jul 1, 2019",NaN,€2.3M,€13K,€4.3M,2019 ~ 2023,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4 ★,2★,Medium,Medium,1 ★,67,62,68,70,69,71,45,63+2,63+2,63+2,66+0,66+0,66+0,66+0,66+0,68+2,68+2,68+2,67+2,70+2,70+2,70+2,67+2,70+2,72+2,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70


In [5]:
#all column names lower case
cols = []
for column in data.columns:
    cols.append(column.lower())    
data.columns = cols
data.head(1)

#underscore space in columns
n_cols = []
for i in range(len(data.columns)):
    n_cols.append(data.columns[i].lower().replace(' ', '_'))
data.columns = n_cols
data.head(1)

#clean weight columng, remove stars from somecolumns, clean height column
data['weight'] = data['weight'].str.rstrip('lbs').astype('int')
data['w/f'] = data['w/f'].str.rstrip('★').astype('int')
data['sm'] = data['sm'].str.rstrip('★').astype('int')
data['ir'] = data['ir'].str.rstrip('★').astype('int')

data['height'] = data['height'].str.replace("'", ".")
data['height'] = data['height'].str.replace('"', "")
data['height'] = data['height'].astype('float')

#clean money columns
data['value'] = data['value'].str.replace("€", "")
data['wage'] = data['wage'].str.replace("€", "")
data['release_clause'] = data['release_clause'].str.replace("€", "")

def money(x):
    if('M' in str(x)):
        x=str(x).replace('M', '')
        x=float(x)*1000000
    elif('K' in str(x)):
        x=str(x).replace('K', '')
        x=float(x)*1000  
    return float(x)

data['value']= data['value'].apply(money)
data['wage']= data['wage'].apply(money)
data['release_clause']= data['release_clause'].apply(money)

In [6]:
#clean all performance values where potential is added as a string 'xx+y'
def findSum(str1):

    temp = "0"
    Sum = 0

    for ch in str1:
        # if current character is a digit
        if (ch.isdigit()):
            temp += ch
        # if current character is an alphabet
        else:
            Sum += int(temp)
            # reset temporary
            temp = "0"
    return Sum + int(temp)

for i in data.columns[-28:-1]: #need to loop over each column otherwise the function doesnt work
    data[i]= data[i].apply(findSum)
    
    data.head(2)

### clean Nans

In [7]:
data.isna().sum()
round(data.isna().sum()/len(data),4)*100  # shows the percentage of null values in a column
nulls_df = pd.DataFrame(round(data.isna().sum()/len(data),2)*100) #division not multiplication like in instructions

nulls_df = nulls_df.reset_index()

nulls_df.columns = ['header_name', 'percent_nulls']
nulls_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
header_name,id,name,age,nationality,club,bp,position,team_&_contract,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
percent_nulls,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#drop Nans
data=data.drop(['loan_date_end'], axis=1) #has 95% NaN's
data=data.dropna(subset=['position','composure'])# drop NaN rows in these columns

In [9]:
data.isna().sum()
round(data.isna().sum()/len(data),4)*100  # shows the percentage of null values in a column
nulls_df = pd.DataFrame(round(data.isna().sum()/len(data),2)*100) #division not multiplication like in instructions

nulls_df = nulls_df.reset_index()

nulls_df.columns = ['header_name', 'percent_nulls']
nulls_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
header_name,id,name,age,nationality,club,bp,position,team_&_contract,height,weight,foot,growth,joined,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
percent_nulls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#delete duplicates
print(data.shape)
data = data.drop_duplicates() 
print(data.shape)

(1915, 100)
(1915, 100)


## Numericals (take same columns as train dataset)

In [11]:
numerical=data.select_dtypes(include=['float64','int64'])
numerical.head(1)

,id,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,ir,pac,sho,pas,dri,def,phy,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,219461,23,6.2,194,7,975000.0,5000.0,0.0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2,2,1,64,30,50,50,66,74,50,50,50,48,47,47,47,48,50,50,50,52,53,53,53,52,61,63,63,63,61,63,69,69,69,63,18,67


In [12]:
keep = (((numerical.corrwith(numerical['ova']))**2)**.5) #transfer negative correlated ones to psotive for later sorting from highest to lowest

In [13]:
#prepare same datframe is the one used formodel training
numerical_corr_clean=numerical.filter(['age','value','wage','reactions','power','shot_power','vision','composure','total_stats','base_stats','pas','dri','phy','ova'], axis=1)
numerical_corr_clean

,age,value,wage,reactions,power,shot_power,vision,composure,total_stats,base_stats,pas,dri,phy,ova
0,23,975000.0,5000.0,63,288,48,39.0,49.0,1551,334,50,50,74,67
1,22,1200000.0,3000.0,64,323,76,64.0,65.0,1726,358,62,74,57,68
2,19,120000.0,500.0,41,171,40,26.0,31.0,808,295,53,55,53,54
3,16,160000.0,500.0,48,242,48,40.0,59.0,1381,303,47,52,59,55
4,24,2300000.0,13000.0,64,337,73,62.0,63.0,1864,407,68,70,71,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,21,325000.0,1000.0,50,245,31,34.0,59.0,1443,315,46,55,57,60
1995,21,190000.0,700.0,59,177,43,36.0,33.0,928,318,57,62,54,59
1996,27,8000000.0,9000.0,73,329,63,81.0,75.0,1867,388,78,77,60,76
1997,34,140000.0,4000.0,61,300,53,56.0,60.0,1639,346,58,58,69,63


In [14]:
#split the data from the target for further modeling
X_num=numerical_corr_clean
#data.select_dtypes(include = np.number)

y= data['ova']
X_num=X_num.drop(['ova'], axis=1)

In [15]:
#load minmax scaler
transformer = load(open('fifa_minmax.pkl', 'rb'))

In [16]:

x_normalized = transformer.transform(X_num)

#x_normalized
#data_norm=pd.DataFrame(x_normalized, columns=X_num.columns)

In [17]:
x_normalized = transformer.transform(X_num)
print(x_normalized.shape)
#x_normalized
data_norm=pd.DataFrame(x_normalized, columns=X_num.columns)

(1915, 13)


## Categoricals (take same columns as train dataset)

In [18]:
categorical=data.select_dtypes(include=['object'])

categorical=categorical.drop(['name','nationality','club','position','team_&_contract','joined','contract','hits'], axis=1)
#use only bp, weight, foot, a/w, d/w
categorical.head(1)

,bp,foot,a/w,d/w
0,CB,Right,Low,High


In [19]:
pd.unique(categorical['a/w'])

array(['Low', 'High', 'Medium'], dtype=object)

In [20]:
#a/w and d/w have some nan as a string not as a nanvalue, very clever trick =)
#they cant be dropped with the dropnan function but need further processing

categorical = categorical.replace('nan', np.NaN, regex=True) #first replace string with Nan


categorical=categorical.dropna(subset=['a/w','d/w'])

In [21]:
#check if created Nan values are gone
categorical.isna().sum()
round(categorical.isna().sum()/len(categorical),4)*100  # shows the percentage of null values in a column
nulls_df = pd.DataFrame(round(categorical.isna().sum()/len(categorical),2)*100) #division not multiplication like in instructions

nulls_df = nulls_df.reset_index()

nulls_df.columns = ['header_name', 'percent_nulls']
nulls_df.T

,0,1,2,3
header_name,bp,foot,a/w,d/w
percent_nulls,0.0,0.0,0.0,0.0


In [22]:
pd.unique(categorical['d/w'])

array(['High', 'Low', 'Medium'], dtype=object)

In [23]:
#load minmax scaler
encoder = load(open('fifa_onehot.pkl', 'rb'))

In [24]:
encoder

OneHotEncoder()

In [25]:
df2=categorical
encoded_for_p = encoder.transform(df2).toarray()
encoded_for_p
encoder.categories_
cols = [colname for row in encoder.categories_ for colname in row]
cols
onehot_encoded_for_p = pd.DataFrame(encoded_for_p, columns=cols)
cols_to_drop = [row[0] for row in encoder.categories_]
cols_to_drop
onehot_encoded_for_p = onehot_encoded_for_p.drop(cols_to_drop,axis=1)
onehot_encoded_for_p.head(2)

,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,Right,Low,Medium,Low,Medium
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [26]:
#concat data
X_regression=pd.concat([data_norm.reset_index(drop=True),onehot_encoded_for_p.reset_index(drop=True)], axis=1)
X_regression.head(1)


,age,value,wage,reactions,power,shot_power,vision,composure,total_stats,base_stats,pas,dri,phy,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,Right,Low,Medium,Low,Medium
0,0.259259,0.010833,0.008929,0.549296,0.498392,0.426829,0.341176,0.440476,0.521297,0.40458,0.367647,0.328358,0.734375,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [27]:
lm = load(open('finalized_model.pkl', 'rb'))
lm

LinearRegression()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_regression, y, test_size=0.99, random_state=42)
X_train.head(1)

,age,value,wage,reactions,power,shot_power,vision,composure,total_stats,base_stats,pas,dri,phy,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,Right,Low,Medium,Low,Medium
1687,0.37037,0.077778,0.096429,0.704225,0.437299,0.512195,0.788235,0.666667,0.679593,0.576336,0.647059,0.865672,0.4375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [29]:
predictions= lm.predict(X_regression)
print('R2=',r2_score(y, predictions))

mse=np.sqrt(mean_squared_error(y,predictions))
print('MSE=',mse)

rmse = math.sqrt(mse)
print('RMSE=',rmse)

mae = mean_absolute_error(y, predictions)
print('MAE=', mae)

R2= 0.9042479275430594
MSE= 2.091471467850097
RMSE= 1.4461920577330305
MAE= 1.6351670679609567
